# Wikipedia Current Events — Data Pull
Fetches the monthly curated event lists from Wikipedia's Current Events Portal
and saves them as structured CSV data for downstream analysis.

## 1. Setup

In [14]:
from _notebook_setup import *

hit_api = False
save_output = True

# Date range — adjust as needed
START_YEAR  = 2016
START_MONTH = 1
END_YEAR    = datetime.now().year
END_MONTH   = datetime.now().month - 1 or 12  # last completed month

## 2. Fetch Data

In [15]:
if hit_api:
    df_events = wikipedia.get_months(
        start_year=START_YEAR,
        start_month=START_MONTH,
        end_year=END_YEAR,
        end_month=END_MONTH,
    )
    if save_output:
        save_data(df=df_events, filename='00_wikipedia_events.csv', subdir='raw_data')
else:
    df_events = load_data(filename='00_wikipedia_events.csv', subdir='raw_data')
    df_events['date'] = pd.to_datetime(df_events['date'])

df_events['category'] = df_events['category'].str.lower()
print(f"\n{len(df_events)} total events across {df_events['date'].dt.to_period('M').nunique()} months")
df_events.head(10)

📂 Loaded: /Users/annebode/dev/selfevidence.github.io/projects/news_tracker/output/raw_data/00_wikipedia_events.csv (54166 rows)

54166 total events across 121 months


,date,year,month,category,sub_topic,description,wiki_links,sources
0,2016-01-01,2016,1,armed conflicts and attacks,Israeli–Palestinian conflict,"A shooting takes place at a pub in Tel Aviv , ...",January 2016 Tel Aviv shooting|Tel Aviv|Israel...,Haaretz
1,2016-01-01,2016,1,arts and culture,Jesus,"A new 28-foot tall statue of Jesus , dubbed ""J...",Jesus|Imo state,USA Today
2,2016-01-01,2016,1,disasters and accidents,Manila,About one thousand houses in Manila's Tondo di...,"Manila|Tondo, Manila|Philippines|New Year's Ev...",AP via CTV News
3,2016-01-01,2016,1,international relations,Deep and Comprehensive Free Trade Area,The EU-Ukraine Free Trade deal officially come...,Deep and Comprehensive Free Trade Area,Radio Free Europe/Radio Liberty
4,2016-01-01,2016,1,law and crime,Two-child policy,"The two-child policy takes effect in China , a...",Two-child policy|China|One-child policy|Commun...,AFP via Channel NewsAsia
5,2016-01-01,2016,1,politics and elections,Ólafur Ragnar Grímsson,"Ólafur Ragnar Grímsson , who has been the Pres...",Ólafur Ragnar Grímsson|President of Iceland|20...,Visir
6,2016-01-02,2016,1,armed conflicts and attacks,Terrorism in India,"Heavily armed terrorists, reportedly members o...",Jaish-e-Mohammed|Indian Air Force|Pathankot|Pu...,DNA
7,2016-01-02,2016,1,armed conflicts and attacks,Mexican Drug War,"Gisela Mota Ocampo , mayor of Temixco in Mexic...",Gisela Mota Ocampo|Temixco|Mexico|Morelos|Gove...,AP via Daily Mail
8,2016-01-02,2016,1,armed conflicts and attacks,Occupation of the Malheur National Wildlife Re...,"Armed militiamen , including members of the Bu...",Militia organizations in the United States|Bun...,Oregon Live
9,2016-01-02,2016,1,law and crime,Sheikh Nimr,The Saudi Arabian government executes 47 peopl...,Saudi Arabia|Sheikh Nimr|Nimr al-Nimr,BBC| The Nation | Indian Express |Reuters


## 3. Explore

In [16]:
# Events per month
df_events.groupby(df_events['date'].dt.to_period('M')).size().rename('event_count')

date
2016-01    357
2016-02    460
2016-03    543
2016-04    662
2016-05    456
          ... 
2025-09    595
2025-10    495
2025-11    495
2025-12    403
2026-01    436
Freq: M, Name: event_count, Length: 121, dtype: int64

In [39]:
# Acceptable categories
acceptable_categories = [
    'armed conflicts and attacks',
    'arts and culture',
    'business and economy',
    'disasters and accidents',
    'health and environment',
    'international relations',
    'law and crime',
    'politics and elections',
    'science and technology',
    'sports'
]

mapping_category = {

        'armed conflict and attacks': 'armed conflicts and attacks',
        'armed attacks and conflicts': 'armed conflicts and attacks',
        'attacks and armed conflicts': 'armed conflicts and attacks',
        'armed conflicts': 'armed conflicts and attacks',
        'armed conflicts and attack': 'armed conflicts and attacks',
        'armed conflict and attack': 'armed conflicts and attacks',
        'armed conflicts and attacks:': 'armed conflicts and attacks',

        'arts': 'arts and culture',
        'arts and culture;': 'arts and culture',
        'arts and cultures': 'arts and culture',
        'culture and media': 'arts and culture',
        'art & literature': 'arts and culture',
        'video games': 'arts and culture',
        'entertainment': 'arts and culture',
        'media': 'arts and culture',
        'art and culture': 'arts and culture',
        'music': 'arts and culture',
        'movies': 'arts and culture',
        'holidays not aligned to the gregorian calendar': 'arts and culture',
        'society': 'arts and culture',
        'other': 'arts and culture',

        'business and economics': 'business and economy',
        'business and economy:': 'business and economy',
        'business  and  economy': 'business and economy',
        'business and finance': 'business and economy',
        'businesses and economy': 'business and economy',
        'business': 'business and economy',

        'disaster and accidents': 'disasters and accidents',
        'disasters and incidents': 'disasters and accidents',
        'disasters and incidents': 'disasters and accidents',
        'natural disasters': 'disasters and accidents',
        'disasters': 'disasters and accidents',

        'health': 'health and environment',
        'health and medicine': 'health and environment',
        'environment': 'health and environment',

        'law and crime:': 'law and crime',
        'crime and law': 'law and crime',
        'law and order': 'law and crime',
        'law and crimes': 'law and crime',
        'crime': 'law and crime',

        'politics and election ': 'politics and elections',
        'politics': 'politics and elections',
        'religion and politics': 'politics and elections',
        'politics and election': 'politics and elections',
        'royalty': 'politics and elections',
        'politics and economics': 'politics and elections',

        'science': 'science and technology',
        'science and research': 'science and technology',
        'science & technology': 'science and technology',
        'science and nature': 'science and technology',
        'science and environment': 'science and technology',
        'transportation': 'science and technology',
        'transport': 'science and technology',

        'sport': 'sports',
        'wrestling news': 'sports',
        'sports:': 'sports',
        'sporting events': 'sports',
        
    }

df_events_clean = df_events.copy(deep=True)
df_events_clean.replace(np.nan, '', inplace=True)
df_events_clean['category'] = df_events_clean['category'].map(mapping_category).fillna(df_events_clean['category'])
save_data(df=df_events_clean, filename='00_wikipedia_events_clean.csv', subdir='processed_data')
# Events per category (overall)
category_counts = df_events_clean['category'].value_counts()

print([c for c in category_counts.index if c not in acceptable_categories])
category_counts

💾 Data saved: /Users/annebode/dev/selfevidence.github.io/projects/news_tracker/output/processed_data/00_wikipedia_events_clean.csv
[]


category
armed conflicts and attacks    11581
health and environment          8679
law and crime                   8105
politics and elections          7100
disasters and accidents         7027
international relations         4681
business and economy            2473
sports                          1835
science and technology          1481
arts and culture                1204
Name: count, dtype: int64

In [28]:
# Browse a specific month
sample_month = '2024-06'
df_events_clean[df_events_clean['date'].dt.to_period('M').astype(str) == sample_month][['date','category','description']]

,date,category,description
44724,2024-06-01,armed conflicts and attacks,Five people and a Hezbollah militant killed an...
44725,2024-06-01,armed conflicts and attacks,The Sudanese Armed Forces bomb a hospital in K...
44726,2024-06-01,armed conflicts and attacks,Around eleven civilians are killed and 42 othe...
44727,2024-06-01,armed conflicts and attacks,Sudan ’s Ambassador to Russia confirms willing...
44728,2024-06-01,armed conflicts and attacks,Russia launches missile and drone strikes acro...
...,...,...,...
45205,2024-06-30,politics and elections,French citizens vote in the first round of leg...
45206,2024-06-30,politics and elections,Uruguayans vote to elect the presidential cand...
45207,2024-06-30,politics and elections,Thousands of Haredi Jewish men protest in Jeru...
45208,2024-06-30,politics and elections,The Bulgarian Orthodox Church elects Metropoli...


In [38]:
df_events_clean[df_events_clean['wiki_links'].str.contains('Minneapolis')]

,date,year,month,category,sub_topic,description,wiki_links,sources
1160,2016-03-20,2016,3,business and economy,Cleveland,The United States -based Sherwin-Williams Comp...,Cleveland|Sherwin-Williams|Minneapolis|Paint|V...,Reuters
1776,2016-04-21,2016,4,arts and culture,Prince (musician),Musician Prince dies at his home at Paisley Pa...,Prince (musician)|Paisley Park Records|Minneap...,News Limited
3026,2016-07-07,2016,7,law and crime,Shooting of Philando Castile,Minnesota Governor Mark Dayton requests a Just...,Governor of Minnesota|Mark Dayton|United State...,The New York Times |CBS News
7329,2017-07-19,2017,7,law and crime,Murder of Justine Damond,Australian Prime Minister Malcolm Turnbull exp...,Prime Minister of Australia|Malcolm Turnbull|M...,ABC News
7359,2017-07-21,2017,7,law and crime,Murder of Justine Damond,"Minneapolis Police Chief Janeé Harteau, upon r...",Minneapolis Chief of Police|Betsy Hodges,ABC News| The New York Times
7478,2017-08-02,2017,8,disasters and accidents,Minnehaha Academy,A natural gas explosion at college prep school...,Minnehaha Academy|Minneapolis|Minnesota,AP|CBS News
9835,2018-03-20,2018,3,law and crime,Shooting of Justine Damond,Minneapolis Police formally charge Mohamed Noo...,Minneapolis Police Department,Star Tribune
13875,2019-01-29,2019,1,disasters and accidents,January 2019 North American cold wave,"Major midwestern cities, including Minneapolis...",Minneapolis|Detroit|Chicago|Milwaukee|Gretchen...,Chicago Tribune | Chicago Sun-Times |WDJT-TV|...
17091,2019-11-27,2019,11,disasters and accidents,Minneapolis,A fire at a Minneapolis apartment building lea...,Minneapolis,The New York Times
19822,2020-05-25,2020,5,law and crime,Murder of George Floyd,George Floyd dies after being restrained by po...,George Floyd|Minneapolis Police Department|Min...,CNN


In [30]:
df_events_clean[df_events_clean['date'] == '2026-01-15']

,date,year,month,category,sub_topic,description,wiki_links,sources
53919,2026-01-15,2026,1,armed conflicts and attacks,Operation Southern Spear,The United States Coast Guard boards and seize...,United States|United States Coast Guard|Guyana...,The Guardian
53920,2026-01-15,2026,1,business and economy,Frigidaire,"Frigidaire issues a recall for 330,000 mini-fr...",Frigidaire|Product recall|Refrigerator,AP
53921,2026-01-15,2026,1,disasters and accidents,2026 Utrecht explosions,At least four people are injured and several b...,Utrecht|Netherlands|Gas explosion,NOS in Dutch |AFP via Al Arabiya|BBC News
53922,2026-01-15,2026,1,disasters and accidents,South Africa,At least 19 people are killed and hundreds are...,South Africa|Provinces of South Africa|Limpopo...,AP
53923,2026-01-15,2026,1,disasters and accidents,Crane (machine),Two people are killed when a construction cran...,Crane (machine)|Rama II Road|Samut Sakhon|Thai...,BBC News|Reuters
53924,2026-01-15,2026,1,international relations,Greenland crisis,French president Emmanuel Macron announces the...,France|President of France|Emmanuel Macron|Fre...,France 24 in French
53925,2026-01-15,2026,1,international relations,Japan–Philippines relations,Japan and the Philippines sign a defense pact ...,Japan|Philippines|Defense pact|Ammunition,AP
53926,2026-01-15,2026,1,international relations,United Arab Emirates–Yemen relations,Faraj Al-Bahsani is dismissed from Yemen 's Pr...,Faraj Al-Bahsani|Yemen|Presidential Leadership...,The New Arab
53927,2026-01-15,2026,1,politics and elections,2026 Ugandan general election,Ugandans vote to elect their president and 529...,Ugandans|President of Uganda|Parliament of Uganda,AP
53928,2026-01-15,2026,1,politics and elections,Singapore,Singaporean prime minister Lawrence Wong resci...,Singapore|Prime Minister of Singapore|Lawrence...,CNA


In [31]:
df_events_clean['sub_topic'].value_counts().head(20)

sub_topic
COVID-19 pandemic                            7636
Russian invasion of Ukraine                   865
Russo-Ukrainian War                           787
Syrian Civil War                              512
Syrian civil war                              460
Israeli–Palestinian conflict                  437
Gaza war                                      410
Israel–Hamas war                              370
War in Afghanistan (2001–2021)                319
War in Afghanistan (2001–present)             292
Somali Civil War (2009–present)               272
Israel–Hezbollah conflict (2023–present)      196
Economic impact of the COVID-19 pandemic      163
European migrant crisis                       162
2022 monkeypox outbreak                       151
Insurgency in Khyber Pakhtunkhwa              137
2016 United States presidential election      135
Impact of the COVID-19 pandemic on sports     129
Kivu conflict                                 128
Boko Haram insurgency                   

## 4. Top 25 Topics — Last 3 Months

Two approaches compared side by side:
- **Option A — Plotly `go.Table`**: consistent with existing chart stack, same iframe embed. No column sorting.
- **Option B — DataTables.js**: sortable columns, live search, pagination. Generated as a self-contained HTML file — embeds identically to Plotly charts. jQuery is already loaded by the minimal-mistakes theme so it costs nothing extra.


In [18]:
# ── Colour palette (shared by both approaches) ───────────────────────────────
CATEGORY_COLORS = {
    'armed conflicts and attacks': '#dc2626',
    'politics and elections':      '#2563eb',
    'law and crime':               '#d97706',
    'disasters and accidents':     '#7c3aed',
    'international relations':     '#0891b2',
    'business and economy':        '#16a34a',
    'science and technology':      '#0d9488',
    'health and environment':      '#65a30d',
    'arts and culture':            '#db2777',
    'sports':                      '#ea580c',
}

# ── Last 3 months ─────────────────────────────────────────────────────────────
latest_date = df_events_clean['date'].max()
cutoff      = latest_date - pd.DateOffset(months=3)

top25 = (
    df_events_clean[
        (df_events_clean['date'] > cutoff) &
        (df_events_clean['sub_topic'].str.strip() != '')
    ]
    .groupby('sub_topic')
    .agg(
        event_count = ('description', 'count'),
        category    = ('category',    lambda x: x.mode()[0]),
        first_seen  = ('date', 'min'),
        last_seen   = ('date', 'max'),
    )
    .sort_values('event_count', ascending=False)
    .head(25)
    .reset_index()
)

top25.insert(0, 'rank', range(1, len(top25) + 1))
top25['period'] = (
    top25['first_seen'].dt.strftime('%b %-d') + ' – ' +
    top25['last_seen'].dt.strftime('%b %-d')
)
top25['color'] = top25['category'].map(CATEGORY_COLORS).fillna('#6b7280')

top25[['rank', 'sub_topic', 'category', 'event_count', 'period']]

,rank,sub_topic,category,event_count,period
0,1,Russo-Ukrainian war (2022–present),armed conflicts and attacks,34,Nov 4 – Jan 29
1,2,Gaza war,armed conflicts and attacks,31,Nov 1 – Jan 31
2,3,Syrian conflict (2024–present),armed conflicts and attacks,21,Nov 14 – Jan 26
3,4,Yemeni civil war (2014–present),armed conflicts and attacks,18,Nov 17 – Jan 29
4,5,War on drugs,armed conflicts and attacks,15,Nov 14 – Jan 26
5,6,2025–2026 Iranian protests,armed conflicts and attacks,14,Dec 31 – Jan 31
6,7,2025 North Indian Ocean cyclone season,disasters and accidents,14,Nov 27 – Dec 7
7,8,Insurgency in Khyber Pakhtunkhwa,armed conflicts and attacks,14,Nov 8 – Jan 29
8,9,Sudanese civil war (2023–present),armed conflicts and attacks,14,Nov 1 – Jan 12
9,10,Operation Southern Spear,armed conflicts and attacks,12,Dec 16 – Jan 23


### Option A — Plotly `go.Table`

In [25]:
row_fills = ['#f8fafc' if i % 2 == 0 else 'white' for i in range(len(top25))]

fig = go.Figure(data=[go.Table(
    columnwidth=[30, 260, 200, 65, 130],
    header=dict(
        values=['<b>#</b>', '<b>Topic</b>', '<b>Category</b>', '<b>Events</b>', '<b>Period</b>'],
        fill_color='#1e293b',
        font=dict(color='white', size=13),
        align=['center', 'left', 'left', 'center', 'center'],
        height=44,
        line_color='#334155',
    ),
    cells=dict(
        values=[
            top25['rank'],
            top25['sub_topic'],
            top25['category'].str.title(),
            top25['event_count'],
            top25['period'],
        ],
        fill_color=[row_fills],
        font=dict(
            size=12,
            color=[
                ['#94a3b8']  * len(top25),     # rank — muted
                ['#0f172a']  * len(top25),     # topic — dark
                top25['color'].tolist(),        # category — coloured
                ['#0f172a']  * len(top25),     # count — dark
                ['#94a3b8']  * len(top25),     # period — muted
            ],
        ),
        align=['center', 'left', 'left', 'center', 'center'],
        height=36,
        line_color='#e2e8f0',
    ),
)])

fig.update_layout(
    title=dict(
        text=(
            f'<b>Top 25 Most Covered Topics</b>'
            f'<span style="font-size:13px; color:#64748b">'
            f'  ·  last 3 months  ·  Wikipedia Current Events</span>'
        ),
        x=0.02, xanchor='left', font=dict(size=18),
    ),
    margin=dict(l=16, r=16, t=56, b=16),
    height=len(top25) * 36 + 100,
    paper_bgcolor='white',
)

save_plotly_figure(fig, filename='00_top25_topics_plotly', for_blog=True)

📝 Blog version saved: /Users/annebode/dev/selfevidence.github.io/docs/assets/charts/news_tracker/00_top25_topics_plotly.html
📊 Plotly figure saved: html: /Users/annebode/dev/selfevidence.github.io/projects/news_tracker/output/figures/00_top25_topics_plotly.html
✨ To embed in Jekyll post, use:
<iframe src="{{ site.baseurl }}/assets/charts/news_tracker/00_top25_topics_plotly.html" width="100%" height="700" frameborder="0"></iframe>


{'html': PosixPath('/Users/annebode/dev/selfevidence.github.io/projects/news_tracker/output/figures/00_top25_topics_plotly.html')}

### Option B — DataTables.js

In [24]:
def save_datatable(df, filename, title, columns, page_length=25, for_blog=True):
    """
    Generate a self-contained DataTables.js HTML file from a DataFrame.

    columns: list of dicts, each with:
        key        — DataFrame column name
        label      — header label
        align      — 'left' | 'center' | 'right'  (default 'left')
        badge_col  — optional: column name holding a hex colour for pill badges
    """
    def _cell(row, col):
        val = str(row[col['key']])
        align = col.get('align', 'left')
        if 'badge_col' in col:
            c = row[col['badge_col']]
            val = (
                f'<span style="background:{c}18;color:{c};padding:2px 9px;'
                f'border-radius:10px;font-size:11px;font-weight:600;'
                f'white-space:nowrap;letter-spacing:.3px">{val}</span>'
            )
        return f'<td style="text-align:{align};vertical-align:middle">{val}</td>'

    rows_html = ''.join(
        '<tr>' + ''.join(_cell(row, col) for col in columns) + '</tr>'
        for _, row in df.iterrows()
    )
    headers_html = ''.join(f'<th>{c["label"]}</th>' for c in columns)

    html = f"""<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <link rel="stylesheet" href="https://cdn.datatables.net/1.13.7/css/jquery.dataTables.min.css">
  <style>
    *, *::before, *::after {{ box-sizing: border-box; }}
    body {{
      font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
      margin: 0; padding: 16px 20px 20px;
      background: #fff; color: #0f172a; font-size: 13px;
    }}
    h3 {{ margin: 0 0 14px; font-size: 15px; font-weight: 700; color: #1e293b; }}
    table.dataTable {{ border-collapse: collapse !important; width: 100% !important; }}
    table.dataTable thead th {{
      background: #1e293b; color: #fff;
      font-size: 12px; font-weight: 600; letter-spacing: .4px; text-transform: uppercase;
      border: none !important; padding: 12px 14px !important;
    }}
    table.dataTable thead th.sorting::after,
    table.dataTable thead th.sorting_asc::after,
    table.dataTable thead th.sorting_desc::after {{ opacity: .7; }}
    table.dataTable tbody tr:nth-child(even) {{ background: #f8fafc; }}
    table.dataTable tbody tr:hover {{ background: #eff6ff !important; transition: background .1s; }}
    table.dataTable tbody td {{
      padding: 9px 14px !important; border-bottom: 1px solid #e2e8f0 !important;
    }}
    .dataTables_wrapper {{  }}
    .dataTables_wrapper .dataTables_filter label,
    .dataTables_wrapper .dataTables_length label {{ font-size: 12px; color: #475569; }}
    .dataTables_wrapper .dataTables_filter input {{
      border: 1px solid #cbd5e1; border-radius: 6px;
      padding: 5px 10px; font-size: 12px; outline: none; margin-left: 6px;
    }}
    .dataTables_wrapper .dataTables_filter input:focus {{
      border-color: #3b82f6; box-shadow: 0 0 0 3px #dbeafe;
    }}
    .dataTables_wrapper .dataTables_length select {{
      border: 1px solid #cbd5e1; border-radius: 6px;
      padding: 4px 8px; font-size: 12px; margin: 0 4px;
    }}
    .dataTables_wrapper .dataTables_info {{ font-size: 12px; color: #64748b; padding-top: 10px; }}
    .dataTables_wrapper .dataTables_paginate {{ padding-top: 8px; }}
    .dataTables_wrapper .dataTables_paginate .paginate_button {{
      font-size: 12px; border-radius: 4px !important; border: none !important; padding: 4px 8px !important;
    }}
    .dataTables_wrapper .dataTables_paginate .paginate_button.current {{
      background: #2563eb !important; color: #fff !important;
    }}
    .dataTables_wrapper .dataTables_paginate .paginate_button:not(.current):hover {{
      background: #eff6ff !important; color: #1d4ed8 !important;
    }}
  </style>
</head>
<body>
  <h3>{title}</h3>
  <table id="dt" class="display" style="width:100%">
    <thead><tr>{headers_html}</tr></thead>
    <tbody>{rows_html}</tbody>
  </table>
  <script src="https://code.jquery.com/jquery-3.7.1.min.js"></script>
  <script src="https://cdn.datatables.net/1.13.7/js/jquery.dataTables.min.js"></script>
  <script>
    $(function() {{
      $('#dt').DataTable({{
        pageLength: {page_length},
        order: [[3, 'desc']],
        language: {{ search: '', searchPlaceholder: 'Search topics…' }},
      }});
    }});
  </script>
</body>
</html>"""

    out_path = FIGURES_DIR / f"{filename}.html"
    out_path.write_text(html, encoding='utf-8')
    print(f"💾 DataTable saved: {out_path}")

    if for_blog:
        blog_path = DOCS_CHARTS_DIR / f"{filename}.html"
        blog_path.write_text(html, encoding='utf-8')
        print(f"📝 Blog version saved: {blog_path}")
        print(f'✨ Embed with:\n<iframe src="{{{{ site.baseurl }}}}/assets/charts/news_tracker/{filename}.html"'
              f' width="100%" height="680" frameborder="0"></iframe>')

    return out_path


save_datatable(
    df=top25,
    filename='00_top25_topics_datatable',
    title='Top 25 Most Covered Topics · last 3 months · Wikipedia Current Events',
    columns=[
        dict(key='rank',        label='#',        align='center'),
        dict(key='sub_topic',   label='Topic',    align='left'),
        dict(key='category',    label='Category', align='left',   badge_col='color'),
        dict(key='event_count', label='Events',   align='center'),
        dict(key='period',      label='Period',   align='center'),
    ],
    page_length=25,
    for_blog=True,
)

💾 DataTable saved: /Users/annebode/dev/selfevidence.github.io/projects/news_tracker/output/figures/00_top25_topics_datatable.html
📝 Blog version saved: /Users/annebode/dev/selfevidence.github.io/docs/assets/charts/news_tracker/00_top25_topics_datatable.html
✨ Embed with:
<iframe src="{{ site.baseurl }}/assets/charts/news_tracker/00_top25_topics_datatable.html" width="100%" height="680" frameborder="0"></iframe>


PosixPath('/Users/annebode/dev/selfevidence.github.io/projects/news_tracker/output/figures/00_top25_topics_datatable.html')